In [ ]:
import cv2
from PIL import Image as PilImage
import time

try:
    # Open the camera stream
    camera = cv2.VideoCapture('http://192.168.2.141/stream')
    
    if not camera.isOpened():
        raise Exception('could not open cam')

    # Read a frame from the video stream
    time.sleep(0.5)
    _, img = camera.grab()
    

    # Check if the image is not None
    if img is not None:
        orig_img = img

        # Convert the OpenCV BGR image to RGB (Pillow uses RGB)
        pil_image = PilImage.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        # Display the image (optional)
        pil_image.show()

        # Save the image to a file (optional)
        pil_image.save("captured_image.jpg")

    else:
        print("The camera returned None image. Check if the camera connection works, if necessary, restart the camera.")

except Exception as e:
    print("An error occurred:", e)

finally:
    # Release the OpenCV video stream
    if camera.isOpened():
        camera.release()

In [ ]:
import cv2
from PIL import Image as PilImage
import threading
import time

class CameraCaptureThread(threading.Thread):
    def __init__(self, camera_url):
        super().__init__()
        self.camera_url = camera_url
        self.capture = cv2.VideoCapture(self.camera_url)
        self.latest_frame = None
        self.running = True

    def run(self):
        while self.running:
            _, frame = self.capture.read()
            if frame is not None:
                self.latest_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                time.sleep(0.1)  # Adjust the delay as needed

    def stop(self):
        self.running = False
        self.capture.release()

try:
    # Create and start the camera capture thread
    camera_thread = CameraCaptureThread('http://192.168.2.141/stream')
    camera_thread.start()

    # Wait for a moment to ensure a frame has been captured
    time.sleep(2)

    # Access the latest captured frame
    latest_frame = camera_thread.latest_frame

    # Check if the frame is not None
    if latest_frame is not None:
        # Convert the OpenCV BGR image to RGB (Pillow uses RGB)
        pil_image = PilImage.fromarray(latest_frame)

        # Display the image (optional)
        pil_image.show()

        # Save the image to a file (optional)
        pil_image.save("captured_image.jpg")

    else:
        print("The camera returned None image. Check if the camera connection works, if necessary, restart the camera.")

finally:
    # Stop the camera capture thread
    camera_thread.stop()
    camera_thread.join()


In [6]:
import cv2
import urllib.request
import numpy as np

# Replace 'your_camera_stream_url' with the actual URL of your HTTP camera stream
camera_stream_url = 'http://192.168.2.141/stream'

# Open the camera stream
stream = urllib.request.urlopen(camera_stream_url)

# Read the still image from the stream
bytes = bytes()

while True:
    bytes += stream.read(1024)
    a = bytes.find(b'\xff\xd8')
    b = bytes.find(b'\xff\xd9')

    if a != -1 and b != -1:
        jpg = bytes[a:b + 2]
        bytes = bytes[b + 2:]

        # Convert the image bytes to a NumPy array
        image_np = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)

        # Display the image
        cv2.imshow('Camera Stream', image_np)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:798: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'
